<a href="https://colab.research.google.com/github/OVP2023/ML/blob/main/simple_%D1%81%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_NN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tqdm import tqdm_notebook

Классификация изображений на датасэте данных CIFAR10 из Keras

In [4]:
import numpy as np
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=10**4, random_state=42)

class_names = np.array(['airplane','automobile ','bird ','cat ','deer ','dog ','frog ','horse ','ship ','truck'])

print (X_train.shape,y_train.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
(40000, 32, 32, 3) (40000, 1)


In [5]:
X_train = X_train / 255
X_val = X_val / 255
X_test = X_test / 255


Реализуйте небольшую свёрточную сеть. Совсем небольшую:

    Входной слой
    Свёртка 3x3 с 10 фильтрами
    Нелинейность на ваш вкус
    Max-pooling 2x2
    Вытягиваем оставшееся в вектор (Flatten)
    Полносвязный слой на 100 нейронов
    Нелинейность на ваш вкус
    Выходной полносвязный слой с softmax

Обучите её так же, как и предыдущую сеть. Если всё хорошо, у вас получится accuracy не меньше 50%.

In [6]:
new_model = tf.keras.models.Sequential([
        tf.keras.Input(shape=X_train.shape[1:]),
        tf.keras.layers.Conv2D(filters=10, kernel_size=(3, 3),padding='same',activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
])

In [7]:
#callbacks  используются для получения метрик обучения и влияния на ход обучения
new_callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.keras',monitor='val_accuracy',save_best_only=True),   # Сохраняется ПРЕДпоследняя лучшая модель с мониторингом по val_accuracy
             tf.keras.callbacks.CSVLogger('training.log'),   #записывает данные обучения по эпохам в CSV файл
             tf.keras.callbacks.EarlyStopping(patience=4)]  #прекращает обучение если patience=7 эпох без улучшения (val_accuracy)

In [11]:
new_model.compile(
    optimizer = tf.keras.optimizers.RMSprop(),
    loss =  'sparse_categorical_crossentropy',                                      #keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['sparse_categorical_accuracy']
    )
new_model.fit(X_train, y_train, batch_size=16, epochs=4, callbacks=new_callbacks, validation_split=0.2)

Epoch 1/4
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 24s 12ms/step - loss: 1.8830 - sparse_categorical_accuracy: 0.3286 - val_loss: 1.4694 - val_sparse_categorical_accuracy: 0.4756
Epoch 2/4
  14/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - loss: 1.4254 - sparse_categorical_accuracy: 0.4862

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - loss: 1.3895 - sparse_categorical_accuracy: 0.5067 - val_loss: 1.3583 - val_sparse_categorical_accuracy: 0.5228
Epoch 3/4
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 42s 11ms/step - loss: 1.2321 - sparse_categorical_accuracy: 0.5676 - val_loss: 1.2435 - val_sparse_categorical_accuracy: 0.5573
Epoch 4/4
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - loss: 1.1339 - sparse_categorical_accuracy: 0.6068 - val_loss: 1.2854 - val_sparse_categorical_accuracy: 0.5552


In [12]:
from sklearn.metrics import accuracy_score

predict_x = new_model.predict(X_test)
classes_x = np.argmax(predict_x,axis=1)

test_acc = accuracy_score(y_test, classes_x)
print("\n Test_acc =", test_acc)
assert test_acc > 0.50, "Not good enough. Back to the drawing board :)"
print(" Not bad!")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step

 Test_acc = 0.5594
 Not bad!
